# PA004 - HEALTH INSURANCE CROS-SELL

## 0 - IMPORTS AND FUNCTIONS

In [1]:
import warnings
import optuna
import random
import pickle

import lightgbm              as lgbm
import scikitplot            as skplt
import matplotlib.patches    as mpatches
import matplotlib.pyplot     as plt
import numpy                 as np
import plotly.express        as px
import pandas                as pd
import seaborn               as sns

from IPython.core.display    import HTML
from IPython.display         import Image
from xgboost                 import XGBClassifier
from sklearn.metrics         import log_loss
from sklearn.model_selection import StratifiedKFold
from optuna.integration      import LightGBMPruningCallback
from imblearn.over_sampling  import RandomOverSampler
from sklearn.metrics import confusion_matrix, accuracy_score,precision_score, roc_auc_score,classification_report,roc_curve,auc

from sklearn                 import preprocessing             as pp
from sklearn                 import model_selection           as ms
from sklearn                 import neighbors                 as nh
from sklearn                 import ensemble                  as en


warnings.filterwarnings( 'ignore' )

C:\Users\perot\anaconda3\envs\health_insurance_venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 0.1 - HELPER FUNCTIONS

In [2]:
def recall_at_k (data, k=20000):
    # Reset Index
    data = data.reset_index( drop=True )

    # Create Ranking Order
    data['ranking'] = data.index + 1

    data['recall_at_k'] = data['response'].cumsum() / data['response'].sum()

    return data.loc[k, 'recall_at_k'] 

def precision_at_k (data, k=20000):
    # Reset Index
    data = data.reset_index( drop=True )

    # Create Ranking Order
    data['ranking'] = data.index + 1

    data['precision_at_k'] = data['response'].cumsum() / data['ranking']

    return data.loc[k, 'precision_at_k']

def cramer_v( x, y):
    cm = pd.crosstab( x, y).values   
    n = cm.sum()
    r, k = cm.shape    
    
    chi2 = ss.chi2_contingency( cm )[0]
    chi2corr = max( 0, chi2 - (k-1)*(r-1)/(n-1) )
    kcorr = k - (k-1)**2/(n-1)
    rcorr = r - (r-1)**2/(n-1) 
    
    return np.sqrt( (chi2corr/n) / ( min( kcorr-1, rcorr-1 ) ) )

def prediction_score(model, data):
    # Separating train and validation dataset for each kfold
    x_train = data.drop(columns=['response'])
    y_train = data['response']
    
    # prediction probability (score)
    yhat_proba = model.predict_proba(x_train)[:, 1].tolist()
    
    # merging score to dataset
    data_scored = data.copy()
    data_scored['score'] = yhat_proba
    
    # sort
    data_scored = data_scored.sort_values('score', ascending=False)
    
    # precision and recall
    data_scored = data_scored.reset_index(drop=True)
    data_scored['n_samples'] = data_scored.index + 1
    data_scored['precision_at_k'] = data_scored['response'].cumsum() / data_scored['n_samples']
    data_scored['recall_at_k'] = data_scored['response'].cumsum() / data_scored['response'].sum()
    
    return data_scored

def cross_validation(kfold, modelName, model, data, at_k):
    # Number of folds
    fold=ms.StratifiedKFold(n_splits = kfold, shuffle=True, random_state=42)
    
    # Performance variables
    precision_list = []
    recall_list = []
    cv_performance = {}
    
    for train_cv,val_cv in fold.split(data, data['response']):
        
        # Separating train and validation dataset for each kfold
        # training data
        x_train_fold = data.iloc[train_cv]
        x_train_fold = x_train_fold[cols_selected]
        
        y_train_fold = data['response'].iloc[train_cv]
        
        # validation data
        x_val_fold = data.iloc[val_cv]
        x_val_fold = x_val_fold[cols_selected]
        
        y_val_fold = data['response'].iloc[val_cv]
        
        # fitting the model
        model_fitted = model.fit(x_train_fold,y_train_fold)
        
        # getting the prediction probability
        x_val_fold['response'] = y_val_fold
        val_scored = prediction_score(model_fitted, x_val_fold)
        
        # Getting precision and recall at k
        precision = val_scored.loc[at_k, 'precision_at_k']
        recall = val_scored.loc[at_k, 'recall_at_k']
        
        precision_list.append(precision)
        recall_list.append(recall)
        
    # calculating the mean and std performance of all kfolds
    precision_cv = np.round(np.mean(precision_list),4).astype(float)
    std_precision_cv = '+/-' + np.round(np.std(precision_list),4).astype(str)
    recall_cv = np.round(np.mean(recall_list),4).astype(float)
    std_recall_cv = ' +/- ' + np.round(np.std(recall_list),4).astype(str)
        
    cv_performance[modelName] = [precision_cv, std_precision_cv, recall_cv, std_recall_cv]
    model_performance_cv = pd.DataFrame(cv_performance, index=['precision_at_k', 'std_precision', 'recall_at_k', 'std_recall'])

    return model_performance_cv

def ml_performance( model_name, precision_atK, recall_atK ):
    
    return pd.DataFrame( {'Model Name': model_name,
                          'Precision_at_K': precision_atK,
                          'Recall_at_K': recall_atK}, index=[0])

def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [18, 8]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container {width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()

In [3]:
jupyter_settings()

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


## 1 - DATAS

In [4]:
# Train Datas - Windows
df = pd.read_csv('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/data/raw/data.csv')

### 1.1 - DATA DESCRIPTION

In [5]:
# Data Dimensions
print( 'Number of Rows: {}'.format( df.shape[0] ) )
print( 'Number of Cols: {}'.format( df.shape[1] ) )

Number of Rows: 381109
Number of Cols: 12


In [ ]:
# Data Types
df.dtypes

In [ ]:
# Changing data types
df['region_code'] = df['region_code'].astype(object)
df['policy_sales_channel'] = df['policy_sales_channel'].astype(object)

In [ ]:
# Check NAN Values
df.isna().sum()

### 1.1 - DATA DESCRIPTIVE

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
# Select columns where the content is different 0 or 1.
num_attributes = df[['age','annual_premium','vintage']]
cat_attributes = df[['driving_license','region_code','policy_sales_channel','previously_insured','gender','vehicle_age','vehicle_damage']]
target_atribute = df['response']

In [ ]:
num_attributes.hist(bins=25, figsize = (10, 5 ));

In [ ]:
# Central Tendency - mean, median
ct1 = pd.DataFrame( num_attributes.apply( np.mean ) ).T
ct2 = pd.DataFrame( num_attributes.apply( np.median ) ).T

# Dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame( num_attributes.apply( np.std ) ).T
d2 = pd.DataFrame( num_attributes.apply( min ) ).T
d3 = pd.DataFrame( num_attributes.apply( max ) ).T
d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min() ) ).T
d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew() ) ).T
d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() ) ).T

# Concatenar
m = pd.concat( [ d2, d3, d4, ct1, ct2, d1, d5, d6 ] ).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m

In [ ]:
# Looking at the distribuition of the following variables.
fig, axs = plt.subplots(3, figsize = (8, 10))
sns.distplot(df['age'], ax=axs[0], bins=10)
sns.distplot(df[df['annual_premium']<100000]['annual_premium'], ax=axs[1])
sns.distplot(df['vintage'], ax=axs[2]);

In [ ]:
fig, axs = plt.subplots(ncols=3, nrows=2, figsize= (15, 8))

sns.countplot(x = 'driving_license', palette = 'Set2', data=df, ax=axs[0][0])
sns.countplot(x = 'previously_insured', palette = 'Set2', data=df, ax=axs[0][1])
sns.countplot(x = 'gender', palette = 'Set2', data=df, ax=axs[0][2])
sns.countplot(x = 'vehicle_age', palette = 'Set2', data=df, ax=axs[1][0])
sns.countplot(x = 'vehicle_damage', palette = 'Set2', data=df, ax=axs[1][1])

In [ ]:
fig = plt.subplots(figsize = (15,5))
sns.countplot(x='region_code', data=df)

In [ ]:
policy_1 = df['policy_sales_channel'].value_counts().iloc[0:55]
policy_1 = policy_1.reset_index()
policy_2 = df['policy_sales_channel'].value_counts().iloc[55:110]
policy_2 = policy_2.reset_index()
policy_3 = df['policy_sales_channel'].value_counts().iloc[110:156]
policy_3 = policy_3.reset_index()

fig, axs = plt.subplots(ncols=3, figsize=(15,15))
sns.barplot(y='index', x='policy_sales_channel', data=policy_1, ax=axs[0], orient='h', order=policy_1['index'])
sns.barplot(y='index', x='policy_sales_channel', data=policy_2, ax=axs[1], orient='h', order=policy_2['index'])
sns.barplot(y='index', x='policy_sales_channel', data=policy_3, ax=axs[2], orient='h', order=policy_3['index'])

## 2 - FEATURE ENGINEERING

In [6]:
df2 = df.copy()

In [7]:
# Fitting vehicle_age column
df2['vehicle_age'] = df2['vehicle_age'].apply(lambda x: 1 if (x == '< 1 Year') else 2 if (x == '1-2 Year') else 3)

# Fitting vehicle_damage column
df2['vehicle_damage'] = df2['vehicle_damage'].apply(lambda x: 1 if (x == 'Yes') else 0)

In [8]:
df2.dtypes

id                       int64
gender                  object
age                      int64
region_code              int64
policy_sales_channel     int64
driving_license          int64
vehicle_age              int64
vehicle_damage           int64
previously_insured       int64
annual_premium           int64
vintage                  int64
response                 int64
dtype: object

### 2.1 - NEW FEATURES

In [9]:
# median_premium_by_region
dict_region_code = df2[['annual_premium', 'region_code']].groupby('region_code').median().to_dict(orient='dict')['annual_premium']
df2['median_premium_by_region'] = df2['region_code'].map(dict_region_code)

# moda policy sales chanel por idade
mode_policy_per_age = df2[['age', 'policy_sales_channel']].groupby('age').agg(pd.Series.mode).to_dict(orient='dict')['policy_sales_channel']
df2['mode_policy_per_age'] = df2['age'].map(mode_policy_per_age)

# Media de carros danificados por idade
avg_carros_danificados_idade = df2[['age', 'vehicle_damage']].groupby('age').mean().to_dict(orient='dict')['vehicle_damage']
df2['avg_vehicle_damage_per_age'] = df2['age'].map(avg_carros_danificados_idade)

# Media de carros danificados por regiao
avg_carros_danificados_regiao = df2[['region_code', 'vehicle_damage']].groupby('region_code').mean().to_dict(orient='dict')['vehicle_damage']
df2['avg_vehicle_damage_region_code'] = df2['region_code'].map(avg_carros_danificados_regiao)

# age_group feature creation
df2['age_group'] = df2['age'].apply(lambda x: 1 if (x >= 18 | x < 30 ) else 2 if (x >= 30 | x < 60 ) else 3)

# vintage_month feature cration
df2['vintage_month'] = round(df2['vintage'] / 31)

# day_premium feature creation
df2['day_premium'] = df2['annual_premium']/df2['vintage']

# age_premium feature cration
df2['age_premium'] = df2['annual_premium']/df2['age']

# Calculating Median Annual Premium by Region_code
premium_rc = df2[['annual_premium', 'region_code']].groupby('region_code').median().to_dict(orient='dict')['annual_premium']
df2['region_premium'] = df2['region_code'].map(premium_rc)

# Calculating Median Aday_premium per policy_sales_channel
day_premium_psc = df2[['day_premium', 'policy_sales_channel']].groupby('policy_sales_channel').mean().to_dict(orient='dict')['day_premium']
df2['avg_day_premium_policy'] = df2['policy_sales_channel'].map(day_premium_psc)

# avg vintage per age
avg_vintage_per_age = df2[['age', 'vintage']].groupby('age').mean().to_dict(orient='dict')['vintage']
df2['avg_vintage_age'] = df2['age'].map(avg_vintage_per_age)

## 3 - DATA FILTERING

In [10]:
df3 = df2.copy()

In [ ]:
df3.head()

## 4 - EXPLORATORY DATA ANALYSIS (EDA)

In [11]:
df4 = df3.copy()

### 4.1 - UNIVARIATE ANALYSIS

In [ ]:
df4.head()

In [ ]:
plt.figure(figsize = (20, 25))
plt.suptitle("Analysis Of Variable Response",fontweight="bold", fontsize=20)

plt.subplot(6,2,1)
sns.countplot(x = 'response', hue = 'gender', palette = 'Set2', data = df4)

plt.subplot(6,2,2)
sns.countplot(x = 'response', hue = 'previously_insured', palette = 'Set2', data = df4)

plt.subplot(6,2,3)
sns.countplot(x = 'response', hue = 'vehicle_age', palette = 'Set2', data = df4)

plt.subplot(6,2,4)
sns.countplot(x = 'response', hue = 'vehicle_damage', palette = 'Set2', data = df4)

plt.subplot(6,2,5)
sns.countplot(x = 'response', hue = 'driving_license', palette = 'Set2', data = df4)

plt.subplot(6,2,6)
sns.countplot(x='response', hue = 'age_group', palette = 'Set2', data=df4)

plt.subplot(6,2,7)
sns.kdeplot(x='age', hue='response', palette = 'Set2', shade=True, data=df4)

plt.subplot(6,2,8)
sns.kdeplot(x='annual_premium', hue='response', palette = 'Set2', shade=True, data=df4)

plt.subplot(6,2,9)
sns.kdeplot(x='day_premium', hue='response', palette = 'Set2', shade=True, data=df4)

plt.subplot(6,2,10)
sns.kdeplot(x='age_premium', hue='response', palette = 'Set2', shade=True, data=df4)

plt.subplot(6,2,11)
sns.kdeplot(x='vintage', hue='response', palette = 'Set2', shade=True, data=df4)

In [ ]:
# 'policy_sales_channel'

# set the figure size
plt.figure(figsize=(25, 10))

# from raw value to percentage
total = df4.groupby('policy_sales_channel')['id'].count().reset_index()
response_1 = df4.loc[df4.response == 1 ].groupby('policy_sales_channel')['id'].count().reset_index()
response_0 = df4.loc[df4.response == 0 ].groupby('policy_sales_channel')['response'].sum().reset_index()
resp = pd.merge(response_1, response_0, how = 'outer', on = 'policy_sales_channel')
resp['id'] = resp['id'].fillna(0)
resp = resp.sort_values(by='policy_sales_channel')
resp['id'] = [i / j * 100 for i,j in zip(resp['id'], total['id'])]
total['id'] = [i / j * 100 for i,j in zip(total['id'], total['id'])]

# bar chart 1 -> top bars (group of 'smoker=No')
bar1 = sns.barplot(x="policy_sales_channel",  y="id", data=total, color='darkblue')

# bar chart 2 -> bottom bars (group of 'smoker=Yes')
bar2 = sns.barplot(x="policy_sales_channel", y="id", data=resp, color='lightblue')

# add legend
plt.xticks(rotation=90)
top_bar = mpatches.Patch(color='darkblue', label='response = No')
bottom_bar = mpatches.Patch(color='lightblue', label='response = Yes')
plt.legend(handles=[top_bar, bottom_bar])

# show the graph
plt.show()

### 4.2 - BIVARIATE ANALYSIS

In [ ]:
#1.The interest on purchase the vehicle insurance is greater for customers that damaged their vehicle before and doesn't have insurance. (FALSE)
d1 = df2[( df2['previously_insured'] == 0) & (df2['vehicle_damage'] == 1)]

ax1 = d1[['response','id']].groupby('response').count().reset_index()
ax1['percentage'] = round(ax1['id'] / d1['id'].count()*100)
ax1

In [ ]:
# 2. The interest on purchase the vehicle insurance is greater for woman than men. (FALSE)
aux2 = pd.crosstab(df2['gender'], df2['response'])
aux2['percentage'] = aux2[1]/(aux2[0]+aux2[1])
aux2

In [ ]:
# 3. The interest on purchase vehicle insurance is greater for vintage customers( 7 months or more )
aux3 = df2[df2['vintage_month'] >= 7 ][['id','response']]
aux4 = df2[df2['vintage_month'] < 7 ][['id','response']]

fix, axs = plt.subplots(ncols = 2, figsize = (15,4))
sns.countplot(x = aux3['response'], ax=axs[0]).set_title('Vintage Customer: 7 months or more')
sns.countplot(x = aux4['response'], ax=axs[1]).set_title('Vintage Customer: 7 months or less')

In [ ]:
aux18 = aux3[['response','id']].groupby('response').count().reset_index()
aux18['percentage'] = round( aux18['id'] / aux3.shape[0] * 100 )
aux18

In [ ]:
aux19 = aux4[['response','id']].groupby('response').count().reset_index()
aux19['percentage'] = round( aux19['id'] / aux4.shape[0] * 100 )
aux19

In [ ]:
# 4. The interest on purchase the vehicle insurance is greater for customers that spend less than 30k for annual premium.
aux5 = df2[df2['annual_premium'] > 30000][['id','response']]
aux6 = df2[df2['annual_premium'] <= 30000][['id','response']]

fix, axs = plt.subplots(ncols = 2, figsize = (15,4))
sns.countplot(x = aux5['response'], ax=axs[0]).set_title('Annual Premium: 30k or more')
sns.countplot(x = aux6['response'], ax=axs[1]).set_title('Annual Premium: 30k or less')

In [ ]:
aux16 = aux5[['response','id']].groupby('response').count().reset_index()
aux16['percentage'] = round( aux16['id'] / aux5.shape[0] * 100 )
aux16

In [ ]:
aux17 = aux6[['response','id']].groupby('response').count().reset_index()
aux17['percentage'] = round( aux17['id'] / aux6.shape[0] * 100 )
aux17

In [ ]:
# 5. The interest on purchase the vehicle insurance is greater for young customers.(Between 18 and 30 years old.)
ax7 = sns.countplot(x = df2['response'], hue=df2['age_group'])

In [ ]:
aux8 = pd.crosstab(df2['age_group'], df2['response'])
aux8['percentage'] = aux8[1]/(aux8[0]+aux8[1])
aux8

In [ ]:
# 6. The interest on purchase the vehicle insurance is greater for customers that have driver license.
aux9 = pd.crosstab(df2['driving_license'], df2['response'])
aux9['percentage'] = round(aux9[1]/(aux9[0]+aux9[1])*100)
aux9

In [ ]:
# 7. The interest on purchase the vehicle insurance is greater for customers that have new cars.
aux10 = pd.crosstab(df2['vehicle_age'], df2['response'])
aux10['percentage'] = round(aux10[1]/(aux10[0]+aux10[1])*100)
aux10

In [ ]:
# 8. The interest on purchase the vehicle insurance is greater for customers that have new cars and have damaged their vehicles.
aux11 = df2[(df2['vehicle_damage'] == 1 )]
aux12 = pd.crosstab(aux11['vehicle_age'], aux11['response'])
aux12['percentage'] = round(aux12[1]/(aux12[0]+aux12[1])*100)
aux12

In [ ]:
# 9. The interest on purchase the vehicle insurance is greater for elderly women.
aux13 = df2[(df2['gender'] == 'Female')]
aux14 = pd.crosstab(aux13['age_group'], aux13['response'])
aux14['percentage'] = round(aux14[1]/(aux14[0]+aux14[1])*100)
aux14

In [ ]:
# 10. The interest on purchase the vehicle insurance is lower for customers that are already insured.
aux15 = pd.crosstab(df2['previously_insured'], df2['response'])
aux15['percentage'] = round(aux15[1]/(aux15[0]+aux15[1])*100)
aux15

**Hypothesis Validation**

1. The interest on purchase the vehicle insurance is greater for customers that damaged their vehicle before and doesn't have insurance.
    **False, of the customers that damaged their car and doesn't have insurance, only 25% show interest in acquire vehicle insurance.**

2. The interest on purchase the vehicle insurance is greater for woman than men.
    **False, only 10% of women show interest in acquire vehicle insurance, whereas 13% of the men show interest on acquire vehicle insurance.**

3. The interest on purchase vehicle insurance is greater for vintage customers ( 7 months or more ).
    **False, the period that customers are on the company doensn't show influency on interest in buying vehicle insurance.**

4. The interest on purchase the vehicle insurance is greater for young customers.(Between 18 and 30 years old.)
    **False, customers that spend more than 30k yearly show greter interest on purchase vehicle insurance.**

5. The interest on purchase the vehicle insurance is greater for young customers.(Between 18 and 30 years old.)
    **False, adults and elderlies show greater interest on buying vehicle insurance.**

6. The interest on purchase the vehicle insurance is greater for customers that have driver license.
    **True, arround 12% of customers that hold a driving license show interest in buying the vehicle insurance.**

7. The interest on purchase the vehicle insurance is greater for customers that have new cars.
    **False, the interest is greater for customers that own an old car.**

8. The interest on purchase the vehicle insurance is greater for customers that have new cars and have damaged their vehicles.
    **False, of the customers who damaged their car, the ones that own a old car show greater interest in buying the vehicle insurance (29%), followed by customers that own used cars (27%).**

9. The interest on purchase the vehicle insurance is greater for elderly women.
    **False, adult women show greater interest in buying the vehicle insurance.**

10. The interest on purchase the vehicle insurance is lower for customers that are already insured.
    **True, less than 1% of customers already insured show interest on purchase the vehicle insurance.**


### 4.3 - MULTIVARIATE ANALYSIS

In [ ]:
correlation = df2.corr().round(2)
plt.figure(figsize = (14,7))
sns.heatmap(correlation, annot = True)

## 5 - DATA PREPARATION

In [47]:
df5 = df4.copy()

### 5.1 - SPLIT DATASET INTO TRAINING AND VALIDATION

In [54]:
X = df5.drop( 'response', axis=1 )
y = df5['response'].copy()

x_training, x_valid, y_training, y_valid = ms.train_test_split( X, y, test_size=0.20 )

df6 = pd.concat([ x_training, y_training ], axis=1 )

#### 5.1.1 - Balacning Trainig Dataset

In [55]:
df6[['id','response']].groupby('response').count().reset_index()

,response,id
0,0,267583
1,1,37304


In [57]:
non_interested=df6[df6['response'] == 1]
interested=df6[df6['response'] == 0][:37304]
df=pd.concat([non_interested,interested])
df7 = df.sample(frac=1,random_state=42)
df7.shape

(74608, 23)

In [58]:
df7[['id','response']].groupby('response').count().reset_index()

,response,id
0,0,37304
1,1,37304


In [59]:
x_training = df7.drop( 'response', axis=1 )
y_training = df7['response'].copy()

In [87]:
df8 = df7.copy()

### 5.2 - STANDARDIZATION

In [ ]:
# Subtrai-se a media e divide-se pelo desvio padrao
ss_ap = pp.StandardScaler()
ss_ag = pp.StandardScaler()
ss_dp = pp.StandardScaler()
ss_dpp = pp.StandardScaler()
ss_mp = pp.StandardScaler()
ss_rp = pp.StandardScaler()

# annual premium
df8['annual_premium'] = ss_ap.fit_transform( df8[['annual_premium']].values )
pickle.dump(ss_ap, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/features/annual_premium_scaler.pkl', 'wb'))

# age_premium
df8['age_premium'] = ss_ag.fit_transform( df8[['age_premium']].values )
pickle.dump(ss_ag, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/features/age_premium_scaler.pkl', 'wb'))

# day_premium
df8['day_premium'] = ss_dp.fit_transform( df8[['day_premium']].values )
pickle.dump(ss_dp, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/features/day_premium_scaler.pkl', 'wb'))

# avg_day_premium_policy
df8['avg_day_premium_policy'] = ss_dpp.fit_transform( df8[['avg_day_premium_policy']].values )
pickle.dump(ss_dpp, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/features/avg_day_premium_policy_scaler.pkl', 'wb'))

# median_premium_by_region
df8['median_premium_by_region'] = ss_mp.fit_transform( df8[['median_premium_by_region']].values )
pickle.dump(ss_mp, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/features/median_premium_by_region_scaler.pkl', 'wb'))

# region_premium
df8['region_premium'] = ss_rp.fit_transform( df8[['region_premium']].values )
pickle.dump(ss_rp, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/features/region_premium_scaler.pkl', 'wb'))

### 5.3 - REESCALING

In [ ]:
mms_age = pp.MinMaxScaler()
mms_vintage = pp.MinMaxScaler()
mms_vintage_age = pp.MinMaxScaler()
mms_vm = pp.MinMaxScaler()
mms_vd_rc = pp.MinMaxScaler()

# Age
df8['age'] = mms_age.fit_transform( df8[['age']].values )
pickle.dump(mms_age, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/features/age_scaler.pkl', 'wb'))

# Vintage
df8['vintage'] = mms_vintage.fit_transform( df8[['vintage']].values )
pickle.dump(mms_vintage, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/features/vintage_scaler.pkl', 'wb'))

# avg_vintage_age
df8['avg_vintage_age'] = mms_vintage_age.fit_transform( df8[['avg_vintage_age']].values )
pickle.dump(mms_vintage_age, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/features/avg_vintage_age_scaler.pkl', 'wb'))

# vintage_month
df8['vintage_month'] = mms_vm.fit_transform( df8[['vintage_month']].values )
pickle.dump(mms_vintage, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/features/vintage_scaler.pkl', 'wb'))

# avg_vehicle_damage_region_code
df8['avg_vehicle_damage_region_code'] = mms_vd_rc.fit_transform( df8[['avg_vehicle_damage_region_code']].values )
pickle.dump(mms_vd_rc, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/features/avg_vehicle_damage_region_code_scaler.pkl', 'wb'))

### 5.4 - ENCODER

In [75]:
# gender
target_encode_gender = df8.groupby( 'gender' )['response'].mean()
df8.loc[:, 'gender'] = df8['gender'].map( target_encode_gender )
#pickle.dump(target_encode_gender, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/features/target_encode_gender_scaler.pkl', 'wb'))

# region_code - Frequency Encoding / Target Encoding / Weighted Targed Encoding
fe_region_code = df8.groupby( 'region_code').size() / len( df8 )
df8.loc[:, 'region_code'] = df8['region_code'].map( fe_region_code )
#pickle.dump(fe_region_code, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/features/frequency_encode_region_code_scaler.pkl', 'wb'))

# policy_sales_channel - Frequency Encoding / Target Encoding
fe_policy_sales_channel = df8.groupby( 'policy_sales_channel' ).size() / len( df8 ) 
df8.loc[:, 'policy_sales_channel'] = df8['policy_sales_channel'].map( fe_policy_sales_channel )
#pickle.dump(fe_policy_sales_channel, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/features/frequency_encode_policy_sales_scaler.pkl', 'wb'))

# vintage_month
target_encode_vintage_month = df8.groupby( 'vintage_month' )['response'].mean()
df8.loc[:, 'vintage_month'] = df8['vintage_month'].map( target_encode_vintage_month )
#pickle.dump(target_encode_vintage_month, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/features/target_encode_vintage_month_scaler.pkl', 'wb'))

### 5.5 - VALIDATION PREPARATION

In [ ]:
# STANDARDIZATION

# annual premium
x_valid.loc[:, 'annual_premium'] = ss_ap.transform( x_valid[['annual_premium']].values )

# age_premium
x_valid.loc[:, 'age_premium'] = ss_ag.transform( x_valid[['age_premium']].values )

# day_premium
x_valid.loc[:, 'day_premium'] = ss_dp.transform( x_valid[['day_premium']].values )

# avg_day_premium_policy
x_valid.loc[:,'avg_day_premium_policy'] = ss_dpp.fit_transform( x_valid[['avg_day_premium_policy']].values )

# median_premium_by_region
x_valid.loc[:,'median_premium_by_region'] = ss_mp.fit_transform( x_valid[['median_premium_by_region']].values )

# region_premium
x_valid.loc[:,'region_premium'] = ss_rp.fit_transform( x_valid[['region_premium']].values )



# REESCALING

# age
x_valid.loc[:, 'age'] = mms_age.transform( x_valid[['age']].values )

# vintage
x_valid.loc[:, 'vintage'] = mms_vintage.transform( x_valid[['vintage']].values )

# avg_vintage_age
x_valid.loc[:,'avg_vintage_age'] = mms_vintage_age.fit_transform( x_valid[['avg_vintage_age']].values )

# vintage_month
x_valid.loc[:,'vintage_month'] = mms_vm.fit_transform( x_valid[['vintage_month']].values )

# avg_vehicle_damage_region_code
x_valid.loc[:,'avg_vehicle_damage_region_code'] = mms_vd_rc.fit_transform( x_valid[['avg_vehicle_damage_region_code']].values )



# ENCODER

# policy sales channel
x_valid.loc[:, 'policy_sales_channel'] = x_valid['policy_sales_channel'].map( fe_policy_sales_channel )

# region code
x_valid.loc[:, 'region_code'] = x_valid.loc[:, 'region_code'].map( fe_region_code )

# gender
x_valid.loc[:, 'gender'] = x_valid.loc[:, 'gender'].map( target_encode_gender )

# vintage_month
x_valid.loc[:, 'vintage_month'] = x_valid['vintage_month'].map( target_encode_vintage_month )

# FILL NAN VALUES
x_valid = x_valid.fillna(0)

## 6 - FEATURE SELECTION

In [ ]:
# model definition
forest = en.ExtraTreesClassifier( n_estimators =250, random_state=0, n_jobs=-1 )

# data preparation
x_train_n = df8.drop( ['id', 'response'], axis=1 )
y_train_n  = y_training.values
forest.fit( x_train_n, y_train_n )

In [ ]:
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0 )
indices = np.argsort(importances)[::-1]

# Print the feature ranking 
print("Feature ranking:")
df = pd.DataFrame()
for i,j in zip( x_train_n, forest.feature_importances_ ):
    aux = pd.DataFrame( {'feature': i, 'importance': j}, index=[0] )
    df = pd.concat( [df,aux], axis = 0 )
    
print( df.sort_values( 'importance', ascending=False ) )

# Plot the impurity-based feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(x_train_n.shape[1]), importances[indices], color="r", yerr=std[indices], align="center")
plt.xticks(range(x_train_n.shape[1]), indices)
plt.xlim([-1, x_train_n.shape[1]])
plt.show()

## 7 - FEATURE ENGINEERING

### 7.1 - MACHINE LEARNING MODEL TRAINING

#### 7.1.1 - UNBALANCED DATASET

In [ ]:
cols_selected = ['day_premium',
'vintage',
'age_premium',
'annual_premium',
'vehicle_damage',
'vintage_month',
'previously_insured',
'policy_sales_channel',
'region_code',
'region_premium',
'median_premium_by_region',
'avg_day_premium_policy',
'age',
'avg_vehicle_damage_per_age',
'vehicle_age',
'avg_vehicle_damage_region_code',
'gender',
'avg_vintage_age',]

x_training = df6[ cols_selected ]
x_validation = x_valid[ cols_selected ]
y_validation = y_valid

#### 7.1.2 - BALANCED DATASET

In [ ]:
cols_selected = ['vehicle_damage',
'previously_insured',
'vintage',
'day_premium',
'age_premium',
'annual_premium',
'vintage_month',
'region_code',
'avg_vehicle_damage_per_age',
'policy_sales_channel',
'age',
'median_premium_by_region',
'region_premium',
'avg_day_premium_policy',
'vehicle_age',
'avg_vehicle_damage_region_code',
'avg_vintage_age',
'gender']
x_training = df8[ cols_selected ]
x_validation = x_valid[ cols_selected ]
y_validation = y_valid

#### 7.1.1 - XGB Model

In [ ]:
# model definition
xgb_model = XGBClassifier()

# model training
xgb_model.fit( x_training, y_training )

# model prediction - The generalization POWER
yhat_xgb = xgb_model.predict_proba( x_validation )

In [ ]:
fig, axs = plt.subplots(ncols= 3, figsize = (18,5))

# cumulative gain - Metric for sorting problem
skplt.metrics.plot_cumulative_gain(y_validation, yhat_xgb, ax=axs[0],title='Cumulative Gain - XGB');

# Lift Curve
skplt.metrics.plot_lift_curve(y_validation, yhat_xgb,ax=axs[1],title='Lift Curve - XGB');

# Roc Curve
skplt.metrics.plot_roc(y_validation, yhat_xgb, ax=axs[2], title='ROC-Curve - XGB');

plt.tight_layout()

#### 7.1.2 - KNN

In [ ]:
# model definition
knn_model = nh.KNeighborsClassifier( n_neighbors=8 )

# model training
knn_model.fit( x_training, y_training )

# model prediction - The generalization POWER
yhat_knn = knn_model.predict_proba( x_validation )

In [ ]:
fig, axs = plt.subplots(ncols= 3, figsize = (18,5))

# cumulative gain - Metric for sorting problem
skplt.metrics.plot_cumulative_gain(y_validation, yhat_knn, ax=axs[0],title='Cumulative Gain - LGBM');

# Lift Curve
skplt.metrics.plot_lift_curve(y_validation, yhat_knn,ax=axs[1],title='Lift Curve - LGBM');

# Roc Curve
skplt.metrics.plot_roc(y_validation, yhat_knn, ax=axs[2], title='ROC-Curve - LGBM');

plt.tight_layout()

#### 7.1.3 - Light gradient Boostin Machine Classifier Model

In [ ]:
# Model Definition
lgbm_model = lgbm.LGBMClassifier( learning_rate=0.09,max_depth=-5,random_state=42 )

# Model training
model_lgbm = lgbm_model.fit( x_training, y_training )

# Model Prediction
yhat_lgbm = model_lgbm.predict_proba( x_validation )

In [ ]:
fig, axs = plt.subplots(ncols= 3, figsize = (18,5))

# cumulative gain - Metric for sorting problem
skplt.metrics.plot_cumulative_gain(y_validation, yhat_lgbm, ax=axs[0],title='Cumulative Gain - LGBM');

# Lift Curve
skplt.metrics.plot_lift_curve(y_validation, yhat_lgbm,ax=axs[1],title='Lift Curve - LGBM');

# Roc Curve
skplt.metrics.plot_roc(y_validation, yhat_lgbm, ax=axs[2], title='ROC-Curve - LGBM');

plt.tight_layout()

In [ ]:
# Model Definition
lgbm_model = lgbm.LGBMClassifier( learning_rate=0.09,max_depth=-5,random_state=42 )

# Model training
model_lgbm = lgbm_model.fit( x_training, y_training )

# Model Prediction
yhat_lgbm1 = model_lgbm.predict( x_validation )

## 8 - MODEL PERFORMANCE

### 8.1 - XGB MODEL PERFORMANCE

In [ ]:
# Copy Data
df8_unbalanced = x_valid.copy()
df8_unbalanced['response'] = y_valid.copy()

# Propensity score
df8_unbalanced['score'] = yhat_xgb[:,1].tolist()

# sorted clients by score
df8_unbalanced = df8_unbalanced.sort_values( 'score', ascending=False )

# Compute precision at K
precision_atK = precision_at_k(df8_unbalanced, k=14000 )

# Compute precision at K
recall_atK = recall_at_k(df8_unbalanced, k=14000 )

xgb_performance = ml_performance('XGB Model', precision_atK, recall_atK )
xgb_performance

### 8.2 - KNN MODEL PERFORMANCE

In [ ]:
# Copy Data
df8_unbalanced = x_valid.copy()
df8_unbalanced['response'] = y_valid.copy()

# Propensity score
df8_unbalanced['score'] = yhat_knn[:,1].tolist()

# sorted clients by score
df8_unbalanced = df8_unbalanced.sort_values( 'score', ascending=False )

# Compute precision at K
precision_atK = precision_at_k(df8_unbalanced, k=14000 )

# Compute precision at K
recall_atK = recall_at_k(df8_unbalanced, k=14000 )

KNN_performance = ml_performance('KNN_Model', precision_atK, recall_atK )
KNN_performance

### 8.3 - LGBM MODEL PERFORMANCE

In [ ]:
# Copy Data
df8_unbalanced = x_valid.copy()
df8_unbalanced['response'] = y_valid.copy()

# Propensity score
df8_unbalanced['score'] = yhat_lgbm[:,1].tolist()

# sorted clients by score
df8_unbalanced = df8_unbalanced.sort_values( 'score', ascending=False )

# Compute precision at K
precision_atK = precision_at_k(df8_unbalanced, k=14000 )

# Compute precision at K
recall_atK = recall_at_k(df8_unbalanced, k=14000 )

lgbm_performance = ml_performance('LGBM Model', precision_atK, recall_atK )
lgbm_performance

In [ ]:
print(classification_report(y_valid,yhat_lgbm1))

### 8.3 - PERFORMANCE FOR ALL MODELS

In [ ]:
model_performance = pd.concat( [xgb_performance, KNN_performance, lgbm_performance] )
model_performance.sort_values('Precision_at_K', ascending= False)

## 9 - HYPERPARAMETER FINE TUNING

### 9.1 - Cross Validation

In [ ]:
# Concatenation train and validation dataset to use in cross-validation
df_cv = x_training.copy()
df_cv['response'] = y_training

df_aux = x_valid.copy()
df_aux['response'] = y_valid

df_cc = pd.concat([df_cv, df_aux])

In [ ]:
# Dictioary with models instantiated
models = { 'KNN': knn_model,
           'XGB Model': xgb_model,
           'lgbm Model': lgbm_model}

# Cross-validated models performance
model_performance = pd.DataFrame()

for key in models.keys():
    performance_cv = cross_validation(5, key, models[key], df_cc, 14000)
    model_performance = pd.concat([model_performance, performance_cv], axis=1)

model_performance.T

### 9.3 - LGBM Fine Tunning

In [ ]:
X = df8.drop( 'response', axis=1 ).reset_index(drop=True)
y = df8[['response']].reset_index(drop=True)

In [ ]:
def objective(trial, X, y):
    param_grid = {
        #"device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1121218)

    cv_scores = np.empty(5)
    
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model = lgbm.LGBMClassifier(objective="binary", **param_grid)
        model.fit( X_train, y_train, eval_set=[(X_test, y_test)], eval_metric="auc", early_stopping_rounds=100, callbacks=[LightGBMPruningCallback(trial, metric='auc')],  # Add a pruning callback 
                 )
        preds = model.predict_proba(X_test)
        cv_scores[idx] = log_loss(y_test, preds)

    return np.mean(cv_scores)

In [ ]:
study = optuna.create_study(direction='maximize' , study_name="LGBM Classifier")
func = lambda trial: objective(trial, X, y)
study.optimize(func, n_trials=20)

In [ ]:
print(f"\tBest value (rmse): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

#### 9.3.1 - LGBM After Fine Tuning

In [ ]:
params = {
'n_estimators': 10000,
'learning_rate': 0.023388717892026647,
'num_leaves': 2400,
'max_depth': 3,
'min_data_in_leaf': 8100,
'lambda_l1': 75,
'lambda_l2': 60,
'min_gain_to_split': 6.871322598066593,
'bagging_fraction': 0.6000000000000001,
'bagging_freq': 1,
'feature_fraction': 0.9}

In [ ]:
# Model Definition
lgbm_model = lgbm.LGBMClassifier(**params )

# Model Training
model_lgbm = lgbm_model.fit( x_training, y_training )

# Model Prediction
yhat_lgbm = model_lgbm.predict_proba( x_validation )

In [ ]:
# Model Definition
lgbm_model = lgbm.LGBMClassifier(n_estimators=10000, learning_rate=0.20184012746410188,is_unbalance=True, objective='binary', metric='auc', boost_from_average=False,
                                 num_leaves=20, max_depth=12, min_data_in_leaf=6500, lambda_l1=25, lambda_l2=90, min_gain_to_split=0.044158872935457116,
                                 bagging_fraction=0.9, bagging_freq=1, feature_fraction=0.9 )

# Model Training
model_lgbm = lgbm_model.fit( x_training, y_training )

# Model Prediction
yhat_lgbm = model_lgbm.predict_proba( x_validation )

In [ ]:
fig, axs = plt.subplots(ncols= 3, figsize = (18,5))

# cumulative gain - Metric for sorting problem
skplt.metrics.plot_cumulative_gain(y_validation, yhat_lgbm, ax=axs[0],title='Cumulative Gain - LGBM');

# Lift Curve
skplt.metrics.plot_lift_curve(y_validation, yhat_lgbm,ax=axs[1],title='Lift Curve - LGBM');

# Roc Curve
skplt.metrics.plot_roc(y_validation, yhat_lgbm, ax=axs[2], title='ROC-Curve - LGBM');

plt.tight_layout()

In [ ]:
# Copy Data
df8_unbalanced = x_valid.copy()
df8_unbalanced['response'] = y_valid.copy()

# Propensity score
df8_unbalanced['score'] = yhat_lgbm[:,1].tolist()

# sorted clients by score
df8_unbalanced = df8_unbalanced.sort_values( 'score', ascending=False )

# Compute precision at K
precision_atK = precision_at_k(df8_unbalanced, k=14000 )

# Compute precision at K
recall_atK = recall_at_k(df8_unbalanced, k=14000 )

lgbm_performance_cv = ml_performance('LGBM Model', precision_atK, recall_atK )
lgbm_performance_cv

In [ ]:
#pickle.dump(lgbm_model, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/models/lgbm_model.pkl', 'wb'))

### 9.4 - XGB FINE TUNING

In [ ]:
import random

In [ ]:
param = {
     'n_estimators': [1000, 1500, 2000, 2500], 
     'eta': [0.01, 0.03],
     'max_depth': [3, 5, 9],
     'subsample': [0.1, 0.5, 0.7],
     'colsample_bytree': [0.3, 0.7, 0.9],
     'min_child_weight':[3, 8, 15] }

MAX_EVAL = 5

In [ ]:
final_result = pd.DataFrame()

for i in range( MAX_EVAL ):
# choose values for parameters randomly
    hp = { k: random.sample(v, 1)[0] for k, v in param.items() }
    print( hp )
    
#    # model
    model_xgb = XGBClassifier(colsample_bytree =hp['colsample_bytree'],
                               subsample = hp['subsample'],
                               objective='binary:logistic', 
                               n_estimators=hp['n_estimators'], 
                               max_depth=hp['max_depth'],
                               min_child_weight = hp['min_child_weight'],
                               eta=hp['eta'])
    # performance
    result = cross_validation(2, 'xgb classifier', xgb_model, df_cc, 20000)
    final_result = pd.concat( [final_result, result] )

final_result

#### 9.4.1 - XGB AFTER TUNNING

In [ ]:
pickle.dump(model_xgb_cv, open('C:/Users/perot/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/src/models/xgb_model.pkl', 'wb'))

In [ ]:
# XGB CROSS VALIDATION

# model definition
model_xgb_cv = XGBClassifier( colsample_bytree = 0.3,
                              subsample = 0.1,
                              objective='binary:logistic', 
                              n_estimators=1500, 
                              max_depth=9,
                              min_child_weight = 3,
                              eta= 0.03 )

# model training
model_xgb_cv.fit( x_training, y_training )

# model prediction - The generalization POWER
yhat_xgb_cv = model_xgb_cv.predict_proba( x_validation )

In [ ]:
# Copy Data
df8_unbalanced = x_valid.copy()
df8_unbalanced['response'] = y_valid.copy()

# Propensity score
df8_unbalanced['score'] = yhat_xgb_cv[:,1].tolist()

# sorted clients by score
df8_unbalanced = df8_unbalanced.sort_values( 'score', ascending=False )

# Compute precision at K
precision_atK = precision_at_k(df8_unbalanced, k=14000 )

# Compute precision at K
recall_atK = recall_at_k(df8_unbalanced, k=14000 )

lgbm_performance_cv = ml_performance('XGB Model', precision_atK, recall_atK )
lgbm_performance_cv

In [ ]:
df8.head()

## 10 - DEPLOYING MODEL TO PRODUCTION

### 10.1 - HEALTHINSURANCE CLASS

In [ ]:
import pickle
import inflection
import numpy  as np
import pandas as pd

class HealthInsurance:
    
    def __init__( self ):
        self.home_path                                = ''
        self.frequency_encode_policy_sales_scaler     = pickle.load( open( self.home_path + 'src/features/frequency_encode_policy_sales_scaler.pkl', 'rb') )
        self.frequency_encode_region_code_scaler      = pickle.load( open( self.home_path + 'src/features/frequency_encode_region_code_scaler.pkl', 'rb') )
        self.target_encode_gender_scaler              = pickle.load( open( self.home_path + 'src/features/target_encode_gender_scaler.pkl', 'rb') )
        self.target_encode_vintage_month_scaler       = pickle.load( open( self.home_path + 'src/features/target_encode_vintage_month_scaler.pkl', 'rb') )
        self.age_scaler                               = pickle.load( open( self.home_path + 'src/features/age_scaler.pkl', 'rb') )
        self.avg_vehicle_damage_region_code_scaler    = pickle.load( open( self.home_path + 'src/features/age_scaler.pkl', 'rb') )
        self.avg_vintage_age_scaler                   = pickle.load( open( self.home_path + 'src/features/avg_vintage_age_scaler.pkl', 'rb') )
        self.vintage_scaler                           = pickle.load( open( self.home_path + 'src/features/vintage_scaler.pkl', 'rb') )
        self.age_premium_scaler                       = pickle.load( open( self.home_path + 'src/features/age_premium_scaler.pkl', 'rb') )
        self.annual_premium_scaler                    = pickle.load( open( self.home_path + 'src/features/age_premium_scaler.pkl', 'rb') )
        self.avg_day_premium_policy_scaler            = pickle.load( open( self.home_path + 'src/features/avg_day_premium_policy_scaler.pkl', 'rb') )
        self.day_premium_scaler                       = pickle.load( open( self.home_path + 'src/features/day_premium_scaler.pkl', 'rb') )
        self.median_premium_by_region_scaler          = pickle.load( open( self.home_path + 'src/features/median_premium_by_region_scaler.pkl', 'rb') )
        self.region_premium_scaler                    = pickle.load( open( self.home_path + 'src/features/region_premium_scaler.pkl', 'rb') )
    
    def data_cleaning( self, df5 ):
 
        ## rename Columns
        cols_old = ['id', 'Gender', 'Age', 'Driving_License', 'Region_Code', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Annual_Premium', 'Policy_Sales_Channel', 'Vintage']

        snakecase = lambda x: inflection.underscore( x )
        cols_new = list( map( snakecase, cols_old ) )
        
        # rename
        df5.columns = cols_new
        
        return( df5 )

    def feature_engineering( self, df5 ):
        
        # Fitting vehicle_age column
        df5['vehicle_age'] = df5['vehicle_age'].apply(lambda x: 1 if (x == '< 1 Year') else 2 if (x == '1-2 Year') else 3)

        # Fitting vehicle_damage column
        df5['vehicle_damage'] = df5['vehicle_damage'].apply(lambda x: 1 if (x == 'Yes') else 0)
        
        # median_premium_by_region
        dict_region_code = df5[['annual_premium', 'region_code']].groupby('region_code').median().to_dict(orient='dict')['annual_premium']
        df5['median_premium_by_region'] = df5['region_code'].map(dict_region_code)

        # moda policy sales chanel por idade
        mode_policy_per_age = df5[['age', 'policy_sales_channel']].groupby('age').agg(pd.Series.mode).to_dict(orient='dict')['policy_sales_channel']
        df5['mode_policy_per_age'] = df5['age'].map(mode_policy_per_age)

        # Media de carros danificados por idade
        avg_carros_danificados_idade = df5[['age', 'vehicle_damage']].groupby('age').mean().to_dict(orient='dict')['vehicle_damage']
        df5['avg_vehicle_damage_per_age'] = df5['age'].map(avg_carros_danificados_idade)

        # Media de carros danificados por regiao
        avg_carros_danificados_regiao = df5[['age', 'region_code']].groupby('age').mean().to_dict(orient='dict')['region_code']
        df5['avg_vehicle_damage_region_code'] = df5['age'].map(avg_carros_danificados_regiao)

        # age_group feature creation
        df5['age_group'] = df5['age'].apply(lambda x: 1 if (x >= 18 | x < 30 ) else 2 if (x >= 30 | x < 60 ) else 3)

        # vintage_month feature cration
        df5['vintage_month'] = round(df5['vintage'] / 31)

        # day_premium feature creation
        df5['day_premium'] = df5['annual_premium']/df5['vintage']

        # age_premium feature cration
        df5['age_premium'] = df5['annual_premium']/df5['age']

        # Calculating Median Annual Premium by Region_code
        premium_rc = df5[['annual_premium', 'region_code']].groupby('region_code').median().to_dict(orient='dict')['annual_premium']
        df5['region_premium'] = df5['region_code'].map(premium_rc)

        # Calculating Median Aday_premium per policy_sales_channel
        day_premium_psc = df5[['day_premium', 'policy_sales_channel']].groupby('policy_sales_channel').mean().to_dict(orient='dict')['day_premium']
        df5['avg_day_premium_policy'] = df5['policy_sales_channel'].map(day_premium_psc)

        # avg vintage per age
        avg_vintage_per_age = df5[['age', 'vintage']].groupby('age').mean().to_dict(orient='dict')['vintage']
        df5['avg_vintage_age'] = df5['age'].map(avg_vintage_per_age)
        
        return( df5 )
    
    def data_preparation( self, df5 ):
              
        # STANDARDIZATION

        # annual premium
        df5['annual_premium'] = self.annual_premium_scaler.transform( df5[['annual_premium']].values )

        # age_premium
        df5['age_premium'] = self.age_premium_scaler.transform( df5[['age_premium']].values )

        # day_premium
        df5['day_premium'] = self.day_premium_scaler.transform( df5[['day_premium']].values )

        # avg_day_premium_policy
        df5['avg_day_premium_policy'] = self.avg_day_premium_policy_scaler.transform( df5[['avg_day_premium_policy']].values )

        # median_premium_by_region
        df5['median_premium_by_region'] = self.median_premium_by_region_scaler.transform( df5[['median_premium_by_region']].values )

        # region_premium
        df5['region_premium'] = self.region_premium_scaler.transform( df5[['region_premium']].values )



        # REESCALING

        # age
        df5['age'] = self.age_scaler.transform( df5[['age']].values )

        # vintage
        df5['vintage'] = self.vintage_scaler.transform( df5[['vintage']].values )

        # avg_vintage_age
        df5['avg_vintage_age'] = self.avg_vintage_age_scaler.transform( df5[['avg_vintage_age']].values )

        # vintage_month
        df5['vintage_month'] = self.vintage_scaler.transform( df5[['vintage_month']].values )

        # avg_vehicle_damage_region_code
        df5['avg_vehicle_damage_region_code'] = self.avg_vehicle_damage_region_code_scaler.transform( df5[['avg_vehicle_damage_region_code']].values )



        # ENCODER

        # policy sales channel
        df5.loc[:, 'policy_sales_channel'] = df5['policy_sales_channel'].map( self.frequency_encode_policy_sales_scaler )

        # region code
        df5.loc[:, 'region_code'] = df5['region_code'].map( self.frequency_encode_region_code_scaler )

        # gender
        df5.loc[:, 'gender'] = df5['gender'].map( self.target_encode_gender_scaler )

        # vintage_month
        df5.loc[:, 'vintage_month'] = df5['vintage_month'].map( self.target_encode_vintage_month_scaler )

        # FILL NAN VALUES
        x_valid = x_valid.fillna(0)
        
        # Feature Selection
        cols_selected = ['vehicle_damage','previously_insured','vintage','day_premium','age_premium','annual_premium','vintage_month','region_code','avg_vehicle_damage_per_age','policy_sales_channel','age','median_premium_by_region','region_premium',
                         'avg_day_premium_policy','vehicle_age','avg_vehicle_damage_region_code','avg_vintage_age','gender']
        
        return df5[cols_selected]
    

    def get_prediction( self, model, original_data, test_data ):
        # prediction
        pred = model.predict_proba( test_data )
        
        # join pred into the original data
        original_data['score'] = pred[:, 1].tolist()
        
        return original_data.to_json( orient='records', date_format='iso' )

### 10.2 - API HANDLER

In [ ]:
import pickle
import pandas as pd
import os
from flask                           import Flask, request, Response
from healthinsurance.healthinsurance import HealthInsurance

# logading model
model = pickle.load( open( 'src/models/xgb_model.pkl', 'rb' ) )
                          
# initialize API
app = Flask( __name__ )

@app.route( '/healthinsurance/predict', methods=['POST'] )
def health_insurance_predict():
    test_json = request.get_json()
    
    if test_json: #there is data
               
        if isinstance( test_json, dict ): # unique example
            test_raw = pd.DataFrame( test_json, index=[0] )
    
        else:
            test_raw = pd.DataFrame( test_json, columns=test_json[0].keys() ) # multiple examples
            
        # Instantiate Rossmann Class
        pipeline = HealthInsurance()

        # data cleaning
        df1 = pipeline.data_cleaning( test_raw )
              
        # feature engineering
        df2 = pipeline.feature_engineering( df1 )

        # Data Preparation
        df3 = pipeline.data_preparation( df2 )
                              
        # prediction
        df_response = pipeline.get_prediction( model, test_raw, df3 )
        
        return df_response
        
    else:
        return Response( '{}', status=200, mimetype='application/json' )

if __name__ == '__main__':
    app.run('0.0.0.0')

#    port = os.environ.get('PORT', 5000)
#    app.run( host='0.0.0.0', port=port )